# **Task A**

In [34]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression_prediction(w, X):
    X = np.array(X)
    return sigmoid(np.dot(X,w)) # finding 'z'

# # testing
# weights = [0.5, -0.35, 0.55]
# features = [1, 4, 6]
# prediction = logistic_regression_prediction(weights, features)
# print("Prediction:", prediction)

Prediction: 0.9168273035060777


# **Task B**

In [33]:
import numpy as np

def logistic_loss(predic, Y):
    Y = np.array(Y)
    predic = np.clip(predic, 1e-10, 1 - 1e-10) #to prevent log(0)
    return -np.mean(Y * np.log(predic) + (1 - Y) * np.log(1 - predic)) #calculating loss

# # testing
# predictions = [0.9, 0.7, 0.1, 0.3]
# actual_values = [1, 0, 0, 1]
# loss_value = logistic_loss(predictions, actual_values)
# print("Logistic Loss:", loss_value)


Logistic Loss: 0.6546666599918811


# **Task C**

In [35]:
import numpy as np

def logistic_loss_gradient(predic, X,Y):
    return np.dot(np.array(X).T, np.array(predic) - np.array(Y)) / len(Y)

# # testing
# feat = [[1, 2], [1, 3], [1, 2], [1, 5]]
# predic = [0.9, 0.7, 0.1, 0.3]
# actual = [1, 0, 0, 1]
# grad = logistic_loss_gradient(predic, feat, actual)
# print("Gradient of Loss:", grad)

Gradient of Loss: [ 0.   -0.35]


# **Task D**

In [36]:
import numpy as np

def gradient_descent(w, X, Y, learning_rate=0.01, iter=1000, tol=1e-6):
    for i in range(iter):
        predic = logistic_regression_prediction(w, X)  #finding predictions
        grad = logistic_loss_gradient(predic, X, Y) #finding gradient
        new_w = w - learning_rate * grad  #updating weights
        weight_change = np.linalg.norm(new_w - w)  #finding change in weight to look for covergence
        w = new_w
        if weight_change < tol:
            print(f"Converged after {i+1} iterations.")
            break
    return w

# # Testing
# X = np.array([[2, 3], [3, 5], [5, 2], [3, 7]])  # feature matrix
# Y = np.array([1, 0, 0, 1])  # labels
# X = np.c_[np.ones(X.shape[0]), X]  # adding bias term
# w = np.array([0.5, -0.35, 0.55])  # initial weights
# print("Initial weights:", w)
# optimized_w = gradient_descent(w, X, Y, learning_rate=0.01, iter=1000, tol=1e-6)
# print("Optimized weights:", optimized_w)

Initial weights: [ 0.5  -0.35  0.55]
Optimized weights: [ 0.67455476 -1.22709782  0.63161033]


# **Task E**

In [37]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris() #loading dataset
X = iris.data #feature matrix
y = iris.target #target matrix
#making feature mean=0 and variance=1
scaler = StandardScaler()
X = scaler.fit_transform(X)

X = np.c_[np.ones(X.shape[0]), X] # adding bias term

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # spliting data into training and validation sets

def one_vs_rest_logistic_regression(X_train, y_train, X_test, y_test, num_classes):
    w_list = []
    accurs = []
    all_predic = []
    for class_label in range(num_classes):
        #converting to b labels
        b_y_train = (y_train == class_label).astype(int)
        b_y_test = (y_test == class_label).astype(int)

        w = np.zeros(X_train.shape[1])
        w = gradient_descent(w, X_train, b_y_train, learning_rate=0.01, iter=1000, tol=1e-6) #performing training using gradient descent
        w_list.append(w) # storing new w
        predic = logistic_regression_prediction(w, X_test)
        all_predic.append(predic) # storing probabilities

        b_predic = (predic >= 0.5).astype(int) #converting probabilities to b
        accur = np.mean(b_predic == b_y_test) #finding current class accur
        accurs.append(accur)
        print(f"Class {class_label} Test Accur: {accur:.4f}")

    all_predic = np.array(all_predic).T
    final_predic = np.argmax(all_predic, axis=1) #making the class with highest probability as final predic
    overall_accur = np.mean(final_predic == y_test) #finding overall accur
    print(f"\nOverall Test Accur: {overall_accur:.4f}")

    return w_list, accurs, overall_accur

num_classes = len(np.unique(y))
w_list, accurs, overall_accur = one_vs_rest_logistic_regression(X_train, y_train, X_test, y_test, num_classes)
for i, (w, accur) in enumerate(zip(w_list, accurs)):
    print(f"Learned w for class {i}: {w}")
    print(f"Accur for class {i}: {accur:.4f}")
print(f"\nOverall Accur: {overall_accur * 100:.4f}%")

Class 0 Test Accur: 1.0000
Class 1 Test Accur: 0.7667
Class 2 Test Accur: 0.9333

Overall Test Accur: 0.9000
Learned w for class 0: [-0.81536848 -0.6380633   0.91583982 -1.04481674 -0.97361716]
Accur for class 0: 1.0000
Learned w for class 1: [-0.66183225  0.05768169 -0.90919961  0.17741897 -0.09469135]
Accur for class 1: 0.7667
Learned w for class 2: [-0.93997667  0.5054276   0.06335685  0.78256361  0.99567327]
Accur for class 2: 0.9333

Overall Accur: 90.0000%
